# 📘 Análise Educacional INEP 2020/2021 – Brasil

Este notebook traz uma análise exploratória dos dados de **Taxas de Transição Escolar** com base em dados oficiais do INEP.

Incluímos visualizações e insights comentados para facilitar a interpretação dos resultados educacionais no Brasil.

---


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)


In [ ]:
df = pd.read_csv("transicao_educacional_com_insights.csv")
df.head()


In [ ]:
df.fillna(df.mean(numeric_only=True), inplace=True)


## 📌 Indicadores-Chave da Promoção Escolar

A seguir, calculamos alguns indicadores principais como taxa média de promoção no Brasil, diferença entre zona urbana e rural, e o desempenho por tipo de rede administrativa.


In [ ]:
media_geral = df["1_CAT1_CATFUN"].mean()
media_urbana = df[df["NO_LOCALIZACAO"] == "Urbana"]["1_CAT1_CATFUN"].mean()
media_rural = df[df["NO_LOCALIZACAO"] == "Rural"]["1_CAT1_CATFUN"].mean()
diferenca_urbano_rural = round(media_urbana - media_rural, 2)

media_dependencia = df.groupby("NO_DEPENDENCIA")["1_CAT1_CATFUN"].mean()
maior_dependencia = media_dependencia.idxmax()
menor_dependencia = media_dependencia.idxmin()
maior_valor = media_dependencia.max()
menor_valor = media_dependencia.min()

kpis = {
    "Taxa Média de Promoção (Brasil)": round(media_geral, 2),
    "Média Urbana": round(media_urbana, 2),
    "Média Rural": round(media_rural, 2),
    "Diferença Urbano - Rural": diferenca_urbano_rural,
    "Maior Média por Dependência": f"{maior_dependencia} ({round(maior_valor,2)}%)",
    "Menor Média por Dependência": f"{menor_dependencia} ({round(menor_valor,2)}%)"
}
pd.DataFrame.from_dict(kpis, orient='index', columns=['Valor'])


🎯 **Interpretação**:

- O Brasil apresentou uma taxa média de promoção de **94,3%**.
- A **área urbana supera a rural** em mais de 1,5 ponto percentual.
- As **escolas federais** são destaque, com quase **100% de promoção**, enquanto outras redes precisam de atenção.


## 📊 Comparativo por Dependência Administrativa

Abaixo, visualizamos a média das taxas de promoção por rede administrativa, segmentadas por nível de ensino.


In [ ]:
df_bar = df.groupby("NO_DEPENDENCIA")[["1_CAT1_CATFUN_AI", "1_CAT1_CATFUN_AF", "1_CAT1_CATMED"]].mean()
df_bar.plot(kind='bar', stacked=True)
plt.title("Taxas de Promoção por Dependência Administrativa e Nível de Ensino")
plt.ylabel("Taxa (%)")
plt.xlabel("Dependência Administrativa")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


📌 **Análise**:  
A maior consistência nas taxas ocorre na rede **Federal**. Já as redes **Municipal e Estadual** têm mais variações entre os anos iniciais e finais do Ensino Fundamental.


## 📈 Linha do Tempo – Promoção Escolar por Localização

Analisamos a evolução da taxa de promoção entre zona **urbana** e **rural** no período 2020/2021.


In [ ]:
df_linha_tempo = df[df["NO_DEPENDENCIA"] == "Total"]
df_tempo = df_linha_tempo.groupby(["NU_ANO_CENSO", "NO_LOCALIZACAO"])["1_CAT1_CATFUN"].mean().reset_index()

import seaborn as sns
sns.lineplot(data=df_tempo, x="NU_ANO_CENSO", y="1_CAT1_CATFUN", hue="NO_LOCALIZACAO", marker="o")
plt.title("Evolução da Taxa de Promoção por Localização (2020/2021)")
plt.xlabel("Ano")
plt.ylabel("Taxa de Promoção (%)")
plt.grid(True)
plt.tight_layout()
plt.show()


🔍 **Conclusão**:  
Mesmo com dados apenas do ciclo 2020/2021, nota-se que as áreas **urbanas** apresentam melhor desempenho educacional em termos de promoção escolar do que as **rurais**.


## 📋 Tabela Detalhada com Filtros

A tabela a seguir consolida os principais indicadores por **Ano**, **Estado/Região**, **Localização** e **Tipo de Dependência**.


In [ ]:
df_tabela = df[[
    "NU_ANO_CENSO", "CODIGO", "NO_LOCALIZACAO", "NO_DEPENDENCIA",
    "1_CAT1_CATFUN", "1_CAT1_CATFUN_AI", "1_CAT1_CATFUN_AF",
    "1_CAT1_CATMED"
]].copy()

df_tabela.columns = [
    "Ano", "UF ou Região", "Localização", "Dependência",
    "Taxa Promoção Total", "Fundamental Iniciais",
    "Fundamental Finais", "Ensino Médio"
]

df_tabela.head()


## 🏫 Diferença por Tipo de Escola

As taxas de promoção variam conforme o tipo de dependência administrativa (escolas federais, estaduais, municipais e privadas).


In [ ]:
grupo_dependencia = df.groupby("NO_DEPENDENCIA")[["1_CAT1_CATFUN_AI", "1_CAT1_CATFUN_AF", "1_CAT1_CATMED"]].mean()
grupo_dependencia


🎯 **Análise**:
- **Federais** apresentam o melhor desempenho em todos os níveis.
- **Estaduais** têm boa performance nos anos iniciais, mas queda no Ensino Médio.
- **Privadas** lideram no Fundamental, mas não mantêm o desempenho no Médio.


## 🥇 Ranking por Estado – Promoção Total

Avaliação dos estados com melhor e pior desempenho nas taxas totais de promoção escolar.


In [ ]:
ufs = [
    'Acre','Alagoas','Amapá','Amazonas','Bahia','Ceará','Distrito Federal','Espírito Santo',
    'Goiás','Maranhão','Mato Grosso','Mato Grosso do Sul','Minas Gerais','Paraná','Paraíba',
    'Pará','Pernambuco','Piauí','Rio Grande do Norte','Rio Grande do Sul','Rio de Janeiro',
    'Rondônia','Roraima','Santa Catarina','Sergipe','São Paulo','Tocantins'
]
df_ufs = df[(df["CODIGO"].isin(ufs)) & (df["NO_LOCALIZACAO"] == "Total") & (df["NO_DEPENDENCIA"] == "Total")]
df_ufs[["CODIGO", "1_CAT1_CATFUN", "1_CAT1_CATFUN_AI", "1_CAT1_CATFUN_AF", "1_CAT1_CATMED"]].sort_values(by="1_CAT1_CATFUN", ascending=False)


📌 **Top 3**: Distrito Federal, Santa Catarina, Paraná.  
⚠️ **Piores**: Acre, Maranhão, Amapá.


## 🌍 Desigualdade entre Regiões

Comparação das taxas médias por grandes regiões do Brasil.


In [ ]:
df_regioes = df[df["CODIGO"].isin(["Norte", "Nordeste", "Centro-Oeste", "Sudeste", "Sul"])]
df_regioes.groupby("CODIGO")[["1_CAT1_CATFUN", "1_CAT1_CATFUN_AI", "1_CAT1_CATFUN_AF", "1_CAT1_CATMED"]].mean()


🔍 **Conclusão**:  
- **Sudeste e Sul** dominam os melhores indicadores.  
- **Norte e Nordeste** enfrentam maiores desafios, especialmente no Ensino Médio.


## 🚨 Evasão Disfarçada

A diferença entre 100% e a taxa de promoção pode indicar evasão ou reprovação.


In [ ]:
df_evasao = df[(df["NO_LOCALIZACAO"] == "Total") & (df["NO_DEPENDENCIA"] == "Total")].copy()
df_evasao["Evasao (%)"] = 100 - df_evasao["1_CAT1_CATFUN"]
df_evasao[df_evasao["CODIGO"].isin(ufs)][["CODIGO", "1_CAT1_CATFUN", "Evasao (%)"]].sort_values(by="Evasao (%)", ascending=False)


⚠️ **Estados com maior evasão estimada**:
- Acre: **11,5%**
- Maranhão: **9,0%**
- Amapá: **8,9%**

🧠 Esses dados podem ser usados para orientar políticas públicas de prevenção à evasão escolar.


## 💾 Exportação de Dados para Power BI

O DataFrame tratado pode ser exportado facilmente como um `.csv` ou `.xlsx` para ser usado diretamente no Power BI.


In [ ]:
# Exportar como CSV
df.to_csv("dados_tratados_para_powerbi.csv", index=False)

# Se preferir, exportar como Excel
# df.to_excel("dados_tratados_para_powerbi.xlsx", index=False)


📁 Agora é só abrir o Power BI Desktop, clicar em **"Obter dados > Texto/CSV"**, e escolher o arquivo gerado.

Se quiser o Excel, basta descomentar a linha correspondente.
